In [63]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from itertools import product

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympdee/sympde'))
from misc.utils import write_lines
from data.pdes import PDEs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
def listify(comb_dict):
    for key, value in comb_dict.items():
        if not isinstance(value, list):
            comb_dict[key] = [value]
    return comb_dict

def generate_combinations(dictionary):
    # Base case: if the dictionary is empty, return an empty list
    if not dictionary:
        return [""]

    # Take one key-value pair from the dictionary
    key, values = next(iter(dictionary.items()))
    remaining_dict = dictionary.copy()
    del remaining_dict[key]

    # Recursively generate combinations for the remaining dictionary
    combinations = generate_combinations(remaining_dict)

    # Generate all possible combinations with the current key-value pair
    output_lines = []
    for value in values:
        for combination in combinations:
            if value is None:
                line = combination
            elif combination == "":
                line = f"--{key} {value}" 
            else:
                if isinstance(value, list):
                    value = " ".join([str(v) for v in value])
                line = f"{combination} --{key} {value}"
            output_lines.append(line)

    return output_lines

def dict_to_array(comb_dict):
    comb_dict = listify(comb_dict)
    output_lines = generate_combinations(comb_dict)
    output_lines = "\n".join(output_lines)
    return output_lines

In [65]:
eps0to1 = np.linspace(0, 1, 3)
eps_0to1s = [[0] + [np.round(eps,1) for eps in epss] for epss in product(eps0to1, eps0to1, eps0to1)]

In [66]:
eps0to1s_nogrid = [[0,0,0,0]]
eps1 = np.linspace(0, 1, 5)[1:]
eps0 = [0]*4
for i in range(1,4):
    for eps1_i in range(len(eps1)):
        eps0_i = eps0.copy()
        eps0_i[i] = eps1[eps1_i]
        eps0to1s_nogrid.append(eps0_i)

eps0to1s_nogrid

[[0, 0, 0, 0],
 [0, 0.25, 0, 0],
 [0, 0.5, 0, 0],
 [0, 0.75, 0, 0],
 [0, 1.0, 0, 0],
 [0, 0, 0.25, 0],
 [0, 0, 0.5, 0],
 [0, 0, 0.75, 0],
 [0, 0, 1.0, 0],
 [0, 0, 0, 0.25],
 [0, 0, 0, 0.5],
 [0, 0, 0, 0.75],
 [0, 0, 0, 1.0]]

In [67]:
pdes = PDEs()
pde_names = [pde for pde in pdes]
pde_names

['Pde1',
 'Pde2',
 'Pde3',
 'Pde4',
 'Pde5',
 'Pde6',
 'Pde7',
 'Pde8',
 'Pde9',
 'Pde10',
 'Pde11',
 'Pde12',
 'Pde13',
 'Pde14',
 'Pde15',
 'Pde16',
 'Pde17',
 'Pde18',
 'Pde19',
 'Pde20',
 'Pde21',
 'Pde22',
 'Pde23',
 'Pde24',
 'KdV']

In [68]:
mlp_hidden_channels = [[100]*n_channels for n_channels in range(1, 10)]
mlp_hidden_channels

[[100],
 [100, 100],
 [100, 100, 100],
 [100, 100, 100, 100],
 [100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100, 100],
 [100, 100, 100, 100, 100, 100, 100, 100, 100]]

In [69]:
mlp_hidden_channels_flat = [[2**16, 2**13, 2**8]]

In [70]:
seeds = [1, 2, 3, 4, 5]

In [71]:
n_train = [1_000, 10_000]
# n_splits_valtest = 5_000
# n_splits = [[n_split_train, n_splits_valtest, n_splits_valtest] for n_split_train in n_splits_train]

In [72]:
lrs = [0.1, 0.01, 0.001]
net = ['TrainP']

In [73]:
batch_sizes = list(reversed([2**i for i in range(1, 10, 2)]))
batch_sizes = list([2**i for i in range(2, 5)])
batch_sizes

[4, 8, 16]

In [74]:

comb_dicts = {
    # 'aug_noaug' : {
    #     'epsilons' : [None, [0, 1., 0.1, 0.4]],
    #     'seed': [1, 2, 3, 4, 5],
    # },
    # 'train' : {
    # },
    # 'eps_0to1_nogrid' : {
    #     'epsilons' : eps0to1s_nogrid,
    #     'seed':  seeds,
    # },
    # 'lconv' : {
    #     'rot_angle' : [0, 10, 45, 90],
    #     'dim':  [4, 9, 16, 25],
    #     'digit' : [None, 0, 5],
    # },
    # 'pdes' : {
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # },
    # 'mlp_hidden_channels' : {
    #     'mlp_hidden_channels' : mlp_hidden_channels,
    #     'pde_name' : ['Pde22', 'Pde24', 'KdV'],
    #     'seed':  seeds,
    # },
    # 'equiv_mag_none' : {
    #     'equiv' : ['mag', 'none'],
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # }
    # 'mlp_hidden_channels_flat' : {
    #     'mlp_hidden_channels' : mlp_hidden_channels_flat,
    #     'pde_name' : pde_names,
    #     'seed':  seeds,
    # }
    # 'transform_types' : {
    #     'transform_type' :  ['space_translation', 'none'],
    #     'seed':  np.arange(1, 101),
    # }
    "sine1d" : {
        'tags' : 'sine1d',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '1 7',
        'eps_mult' : '0. 0. 1. 0.',
        'data_dir' : '../data/sine1d',
        'net' : 'TrainP',
        'seed':  seeds,
        'n_train' :  n_train,
        'lr' : lrs,
    },
    "sine1d-predict" : {
        'tags' : 'sine1d-predict',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '1 7',
        'eps_mult' : '0. 0. 1. 0.',
        'data_dir' : '../data/sine1d',
        'net' : ['Predict-NoneP', 'Predict-CalculatedP', 'Predict-TrainedP'],
        'seed':  seeds,
        'n_train' :  '400',
        'lr' : '0.001'
    },
    "sine2d" : {
        'tags' : 'sine2d',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/sine2d',
        'net' : 'TrainP',
        'n_train' :  '10000',
        'lr' : '0.001',
        'seed':  seeds,
    },
    "sine2d-predict" : {
        'tags' : 'sine2d',
        'batch_size' : '2',
        'max_epochs' : '50',
        'n_test' : '1000',
        'n_val' : '1000',
        'y_high' : '3',
        'y_low' : '1',
        'noise_std' : '0.5',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/sine2d',
        'net' : ['Predict-NoneP', 'Predict-CalculatedP', 'Predict-TrainedP'],
        'n_train' :  '1000',
        'lr' : '0.001',
        'seed':  seeds,
    },
    "mnist" : {
        # 'logger' : 'None',
        'tags' : 'mnist hparams',
        'n_test' : '1000',
        'n_val' : '1000',
        'noise_std' : '0.',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/MNIST',
        'max_epochs' : '100',
        'net' : 'TrainP',
        'n_train' :  '10000',
        'batch_size' : batch_sizes,
        'lr' : lrs,
    },
    "mnist-predict" : {
        # 'logger' : 'None',
        'tags' : 'mnist-predict hparams2',
        'n_test' : '1000',
        'n_val' : '1000',
        'noise_std' : '0.0',
        'grid_size' : '7 7',
        'eps_mult' : '0. 0. 1. 1.',
        'data_dir' : '../data/MNIST',
        # 'net' : 'Predict-NoneP',
        'batch_size' : batch_sizes,
        'net' : ['Predict-NoneP', 'Predict-TrainedP'],
        'n_train' :  '10000',
        'criterion' : 'ce',
        'lr' : lrs,
    },
}


array_dir = '../jobs/arrays/'
for file_name, comb_dict in comb_dicts.items():
    output_file = os.path.join(array_dir, file_name + '.txt')
    output_lines = dict_to_array(comb_dict)
    print(f'### {file_name} ###', comb_dict, output_lines, '\n\n',sep='\n')
    write_lines(output_file, output_lines)


### sine1d ###
{'tags': ['sine1d'], 'batch_size': ['2'], 'max_epochs': ['50'], 'n_test': ['1000'], 'n_val': ['1000'], 'y_high': ['3'], 'y_low': ['1'], 'noise_std': ['0.5'], 'grid_size': ['1 7'], 'eps_mult': ['0. 0. 1. 0.'], 'data_dir': ['../data/sine1d'], 'net': ['TrainP'], 'seed': [1, 2, 3, 4, 5], 'n_train': [1000, 10000], 'lr': [0.1, 0.01, 0.001]}
--lr 0.1 --n_train 1000 --seed 1 --net TrainP --data_dir ../data/sine1d --eps_mult 0. 0. 1. 0. --grid_size 1 7 --noise_std 0.5 --y_low 1 --y_high 3 --n_val 1000 --n_test 1000 --max_epochs 50 --batch_size 2 --tags sine1d
--lr 0.01 --n_train 1000 --seed 1 --net TrainP --data_dir ../data/sine1d --eps_mult 0. 0. 1. 0. --grid_size 1 7 --noise_std 0.5 --y_low 1 --y_high 3 --n_val 1000 --n_test 1000 --max_epochs 50 --batch_size 2 --tags sine1d
--lr 0.001 --n_train 1000 --seed 1 --net TrainP --data_dir ../data/sine1d --eps_mult 0. 0. 1. 0. --grid_size 1 7 --noise_std 0.5 --y_low 1 --y_high 3 --n_val 1000 --n_test 1000 --max_epochs 50 --batch_size 2